### Disclaimer

This closely follows Jonathan Katz' Hybrid Search blog post which can be found here: (https://jkatz05.com/post/postgres/hybrid-search-postgres-pgvector/)[https://jkatz05.com/post/postgres/hybrid-search-postgres-pgvector/]

In [138]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import psycopg2
import json
from tqdm import tqdm
import torch

torch.cuda.is_available()


model = SentenceTransformer('ibm-granite/granite-embedding-278m-multilingual', device='cuda')

def embed_texts(texts, model, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        output = model.encode(batch, convert_to_tensor=True).cpu()
        embeddings.extend(output)
    return embeddings


In [97]:
conn = psycopg2.connect(
    host="localhost",
    database="air",
    user="postgres",
    password="postgres",
    port="5432"
)


In [98]:
with open("docs.json", "r") as f:
    paper_chunks = json.load(f)

# Group chunks by paper ID
paper_dict = {}
for entry in paper_chunks:
    uid = entry["cord_uid"]
    paper_dict.setdefault(uid, []).append(entry["text"])

df = pd.read_csv("../X_Data/subtask4b_query_tweets_train.tsv", sep="\t")
dev_df = pd.read_csv("../X_Data/subtask4b_query_tweets_dev.tsv", sep="\t")

In [126]:
with conn.cursor() as cursor:
    # Enable vector extension
    cursor.execute("CREATE EXTENSION IF NOT EXISTS vector;")

    cursor.execute("DROP TABLE IF EXISTS embeddings;")

    # Create Table if it doesn't exist
    cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS embeddings (
            id SERIAL PRIMARY KEY,
            paper_id TEXT NOT NULL,
            paper_text TEXT NOT NULL,
            embedding vector({model.get_sentence_embedding_dimension()}) NOT NULL
        )
    """)
    conn.commit()

In [139]:
# Get paper embeddings

paper_embeddings = []
for uid, texts in tqdm(paper_dict.items()):
    # Join the text chunks into a single string
    text = " ".join(texts)
    # Embed the text
    embedding = model.encode(text)
    paper_embeddings.append((uid, text, embedding))




  0%|          | 0/7718 [00:00<?, ?it/s]


RuntimeError: HIP error: invalid device function
HIP kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing AMD_SERIALIZE_KERNEL=3
Compile with `TORCH_USE_HIP_DSA` to enable device-side assertions.


In [128]:
with conn.cursor() as cursor:
    cursor.execute("DELETE FROM embeddings")

In [129]:
# Insert paper text and embeddings into the database
with conn.cursor() as cursor:
    for uid, text, embedding in tqdm(paper_embeddings):
        # Convert the embedding to a list
        embedding_list = embedding.tolist()
        # Insert into the database
        cursor.execute("""
            INSERT INTO embeddings (paper_id, paper_text, embedding)
            VALUES (%s, %s, %s)
            """, (uid, text, embedding_list))
    # HNSW index for faster similarity search
   
        
    conn.commit()

100%|██████████| 7718/7718 [00:12<00:00, 603.99it/s]


In [130]:
with conn.cursor() as cursor:
    cursor.execute("""
        SELECT COUNT(*) FROM embeddings
    """)
    count = cursor.fetchone()[0]
    print(f"Total number of rows in the embeddings table: {count}")

    

Total number of rows in the embeddings table: 7718


In [131]:
with conn.cursor() as cursor:
    cursor.execute("""
                   CREATE INDEX IF NOT EXISTS paper_hnsw_idx ON embeddings USING hnsw(embedding vector_cosine_ops) WITH (M=16, ef_construction=200)
                   """)
    # GIN index for full-text search
    cursor.execute("""
                   CREATE INDEX IF NOT EXISTS paper_ft_idx ON embeddings USING GIN (to_tsvector('english', paper_text))
                   """)

    conn.commit()

In [132]:
tweet_embeddings = embed_texts(dev_df["tweet_text"].tolist(), model)

100%|██████████| 44/44 [00:27<00:00,  1.61it/s]


In [133]:
hybrid_search_query = """
WITH
-- Vector search top 100
vector_results AS (
  SELECT
    id,
    paper_id,
    paper_text,
    ROW_NUMBER() OVER (ORDER BY embedding <#> %(query_embedding)s::vector) AS vector_rank
  FROM embeddings
  ORDER BY embedding <#> %(query_embedding)s::vector
  LIMIT 100
),

-- Full-text search top 100
text_results AS (
  SELECT
    id,
    paper_id,
    paper_text,
    ROW_NUMBER() OVER (
      ORDER BY ts_rank_cd(to_tsvector('english', paper_text), websearch_to_tsquery('english', %(query_text)s)) DESC
    ) AS text_rank
  FROM embeddings
  WHERE websearch_to_tsquery('english', %(query_text)s) @@ to_tsvector('english', paper_text)
  LIMIT 100
),

-- Combine results
combined AS (
  SELECT
    COALESCE(v.id, t.id) AS id,
    COALESCE(v.paper_id, t.paper_id) AS paper_id,
    COALESCE(v.paper_text, t.paper_text) AS paper_text,
    COALESCE(v.vector_rank, 1000) AS vector_rank,
    COALESCE(t.text_rank, 1000) AS text_rank,
    (1.0 / (60 + COALESCE(v.vector_rank, 1000)) +
     1.0 / (60 + COALESCE(t.text_rank, 1000))) AS rrf_score
  FROM vector_results v
  FULL OUTER JOIN text_results t ON v.id = t.id
),

-- Choose the top chunk per paper
ranked_chunks AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY paper_id ORDER BY rrf_score DESC) AS per_paper_rank
  FROM combined
)

-- Get top 5 unique papers
SELECT paper_id, paper_text
FROM ranked_chunks
WHERE per_paper_rank = 1
ORDER BY rrf_score DESC
LIMIT 5;
"""

In [ ]:
# Convenience function when code errors in transaction
with conn.cursor() as cursor:
    cursor.execute("ROLLBACK;")

In [134]:
# Get results for first 5 tweets
# Get the top 5 results for each tweet and put them in a list
# with the tweet id, cord_uid (gold paper) and the retrieved paper ids
all_results = []

for i, (tweet_id, tweet_text, gold_paper) in tqdm(enumerate(zip(dev_df["post_id"].tolist(), dev_df["tweet_text"].tolist(), dev_df["cord_uid"].tolist()))):
    # Get the embedding for the tweet
    tweet_embedding = tweet_embeddings[i].tolist()
    
    # Execute the hybrid search query
    with conn.cursor() as cursor:
        cursor.execute(hybrid_search_query, {
            'query_embedding': tweet_embedding,
            'query_text': tweet_text
        })
        results = cursor.fetchall()
        # Add the results to the list
        all_results.append({
            "tweet_id": tweet_id,
            "gold_paper": gold_paper,
            "retrieved_papers": results
        })

1400it [00:37, 37.71it/s]


In [135]:
# Calculate MRR5

def calculate_mrr5(results):
    mrr5 = 0.0
    for result in results:
        gold_paper = result["gold_paper"]
        retrieved_papers = [paper[0] for paper in result["retrieved_papers"]]
        # Find the rank of the gold paper
        if gold_paper in retrieved_papers:
            rank = retrieved_papers.index(gold_paper) + 1
            mrr5 += 1.0 / rank
    print("Results", len(results))
    print("MRR5", mrr5)

    mrr5 /= len(results)
    print("MRR5", mrr5)
    return mrr5

mrr5 = calculate_mrr5(all_results)

print(f"MRR@5: {mrr5:.4f}")

Results 1400
MRR5 803.4500000000006
MRR5 0.5738928571428575
MRR@5: 0.5739


In [136]:

# Get top 5 results from vector search

vector_results = []

for i, (tweet_id, tweet_text, gold_paper) in tqdm(enumerate(zip(dev_df["post_id"].tolist(), dev_df["tweet_text"].tolist(), dev_df["cord_uid"].tolist()))):
    # Get the embedding for the tweet
    tweet_embedding = tweet_embeddings[i].tolist()
    
    # Execute the vector search query
    with conn.cursor() as cursor:
        cursor.execute("""
        WITH paper_rank AS (
  SELECT
    id,
    paper_id,
    paper_text,
    ROW_NUMBER() OVER (ORDER BY embedding <#> %(query_embedding)s::vector) AS vector_rank
  FROM embeddings
  ORDER BY embedding <#> %(query_embedding)s::vector
  )
           
            SELECT paper_id, paper_text
            FROM paper_rank
            WHERE vector_rank = 1
            LIMIT 5
        """, {"query_embedding": tweet_embedding})
        results = cursor.fetchall()
        # Add the results to the list
        vector_results.append({
            "tweet_id": tweet_id,
            "gold_paper": gold_paper,
            "retrieved_papers": results
        })



0it [00:00, ?it/s]

1400it [00:35, 39.78it/s]


In [137]:
mrr5 = calculate_mrr5(vector_results)

print(f"MRR@5 (Vector Search): {mrr5:.4f}")

Results 1400
MRR5 718.0
MRR5 0.5128571428571429
MRR@5 (Vector Search): 0.5129


In [117]:
# Get top 5 results from text search
text_results = []

for i, (tweet_id, tweet_text, gold_paper) in tqdm(enumerate(zip(dev_df["post_id"].tolist(), dev_df["tweet_text"].tolist(), dev_df["cord_uid"].tolist()))):
    # Get the embedding for the tweet
    tweet_embedding = tweet_embeddings[i].tolist()
    
    # Execute the text search query
    with conn.cursor() as cursor:
        cursor.execute("""
        WITH paper_rank AS (
  SELECT
    id,
    paper_id,
    paper_text,
    ROW_NUMBER() OVER (
      ORDER BY ts_rank_cd(to_tsvector('english', paper_text), websearch_to_tsquery('english', %(query_text)s)) DESC
    ) AS text_rank
  FROM embeddings
  WHERE websearch_to_tsquery('english', %(query_text)s) @@ to_tsvector('english', paper_text)
  )
           
            SELECT paper_id, paper_text
            FROM paper_rank
            WHERE text_rank = 1
            LIMIT 5
        """, {"query_text": tweet_text})
        results = cursor.fetchall()
        # Add the results to the list
        text_results.append({
            "tweet_id": tweet_id,
            "gold_paper": gold_paper,
            "retrieved_papers": results
        })



1400it [00:01, 1068.39it/s]


In [118]:
mrr5 = calculate_mrr5(text_results)
print(f"MRR@5 (Text Search): {mrr5:.4f}")

Results 1400
MRR5 33.0
MRR5 0.023571428571428573
MRR@5 (Text Search): 0.0236


In [120]:
print(all_results[:5])
print(vector_results[:5])
print(text_results[:5])

[{'tweet_id': 16, 'gold_paper': '3qvh482o', 'retrieved_papers': [('hg3xpej0', "BACKGROUND: There is growing concern about possible cognitive consequences of COVID-19, with reports of ‘Long COVID’ symptoms persisting into the chronic phase and case studies revealing neurological problems in severely affected patients. However, there is little information regarding the nature and broader prevalence of cognitive problems post-infection or across the full spread of disease severity. METHODS: We sought to confirm whether there was an association between cross-sectional cognitive performance data from 81,337 participants who between January and December 2020 undertook a clinically validated web-optimized assessment as part of the Great British Intelligence Test, and questionnaire items capturing self-report of suspected and confirmed COVID-19 infection and respiratory symptoms. FINDINGS: People who had recovered from COVID-19, including those no longer reporting symptoms, exhibited significa